<a href="https://colab.research.google.com/github/jackiekuen2/recommender-systems/blob/main/Kaggle_The_Movie_Dataset_Content_based_Recommender_in_cast_keywords_director_genres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credits
- https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a
- https://www.kaggle.com/general/74235

In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

 95% 217M/228M [00:01<00:00, 153MB/s]
100% 228M/228M [00:01<00:00, 128MB/s]


In [5]:
!unzip \*.zip  && rm *.zip


Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [6]:
import pandas as pd

In [7]:
# Load movie metadata
metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Top 5 movies by Revnue
metadata.sort_values(by='revenue', ascending=False).head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
14551,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.071,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2.787965e+09,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114.0
26555,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,31.626,/weUSwMdQIa3NaXVzwUoIIcAi85d.jpg,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Tru...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-12-15,2.068224e+09,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.5,7993.0
1639,False,NaN,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.titanicmovie.com,597,tt0120338,en,Titanic,"84 years later, a 101-year-old woman named Ros...",26.8891,/kHXEpyfl6zqn8a6YuozZUujufXf.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1997-11-18,1.845034e+09,194.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.5,7770.0
17818,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,89.8876,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-04-25,1.519558e+09,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0
25084,False,"{'id': 328, 'name': 'Jurassic Park Collection'...",150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.jurassicworld.com/,135397,tt0369610,en,Jurassic World,Twenty-two years after the events of Jurassic ...,32.7905,/jjBgi2r5cRt36xF6iNUEhzscEcb.jpg,"[{'name': 'Universal Studios', 'id': 13}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-06-09,1.513529e+09,124.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The park is open.,Jurassic World,False,6.5,8842.0


In [9]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

# II. Content-based Recommender in actors, directors, genres and plot keywords
- Based on: top 3 actors, director, genres, movie plot keywords

In [10]:
# cast dataset
credits = pd.read_csv('credits.csv')

# plot keywords
keywords = pd.read_csv('keywords.csv')

In [11]:
# Remove bad IDs
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to integers
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into metadata as main dataset
metadata = metadata.merge(right=credits, on='id')
metadata = metadata.merge(right=keywords, on='id')

In [12]:
# Preview
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [13]:
%%time
# Parse the stringified features
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

CPU times: user 33.5 s, sys: 1.16 s, total: 34.7 s
Wall time: 34.7 s


## Function: Top 3

In [14]:
import numpy as np

def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return np.nan

# Extract top 3 elements
def get_tops(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return [] # Return empty list in case of missing information

In [15]:
%%time
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(get_tops)

CPU times: user 807 ms, sys: 6.8 ms, total: 814 ms
Wall time: 813 ms


In [16]:
# Preview
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[Comedy]


## Cleansing
- lowercase
- strip all spaces

In [17]:
def clean_data(x):
    # cast, keywords, genres are list
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        # If director doesn't exist, return empty list
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [18]:
%%time
features = ['cast', 'keywords', 'genres', 'director']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

CPU times: user 387 ms, sys: 1.08 ms, total: 388 ms
Wall time: 387 ms


## Create metadata soup
- A string that contains all the metadata that is used to feed to vectorizer

In [19]:
def create_soup(x):
    return " ".join(x['keywords']) + " " + " ".join(x['cast']) + " " + x['director'] + " " + " ".join(x['genres'])

In [20]:
%%time
metadata['soup'] = metadata.apply(create_soup, axis=1)

CPU times: user 1.04 s, sys: 845 µs, total: 1.04 s
Wall time: 1.04 s


In [21]:
metadata['soup'][:10]

0    jealousy toy boy tomhanks timallen donrickles ...
1    boardgame disappearance basedonchildren'sbook ...
2    fishing bestfriend duringcreditsstinger walter...
3    basedonnovel interracialrelationship singlemot...
4    baby midlifecrisis confidence stevemartin dian...
5    robbery detective bank alpacino robertdeniro v...
6    paris brotherbrotherrelationship chauffeur har...
7     jonathantaylorthomas bradrenfro rachaelleighc...
8    terrorist hostage explosive jean-claudevandamm...
9    cuba falselyaccused secretidentity piercebrosn...
Name: soup, dtype: object

## Count Vectorizer
- you do not want to down-weight the actor/director's presence if he or she has acted or directed in relatively more movies
- The major difference between CountVectorizer() and TF-IDF is the inverse document frequency (IDF) component which is present in later and not in the former

In [22]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

CPU times: user 888 ms, sys: 27.2 ms, total: 916 ms
Wall time: 1.01 s


In [23]:
count_matrix.shape

(46628, 73881)

## Cosine similarity

In [24]:
%%time
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.metrics.pairwise import linear_kernel


cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
# cosine_sim2 = linear_kernel(count_matrix, count_matrix)

CPU times: user 11.7 s, sys: 11.4 s, total: 23.1 s
Wall time: 23.1 s


## Get top recomendations

In [25]:
# Reset index
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [26]:
# top 10 movies with similar descriptions
def get_recommendations(title, cosine_sim, topk=10):
    # Get the index of target movie
    idx = indices[title]
    # Get pairwise similariy scores of all movies that are similar to target movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Reversed-Sort the sim scores
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    # Get the top k, except the first one (the movie itself)
    sim_scores = sim_scores[1:topk]
    movie_indices = [i[0] for i in sim_scores]
    # return pd.DataFrame(metadata['title'].iloc[movie_indices])
    rec_results = pd.DataFrame(metadata['title'].iloc[movie_indices])
    rec_results['sim_scores'] = [i[1] for i in sim_scores]
    return rec_results

In [28]:
get_recommendations('Jurassic World', cosine_sim2)

,title,sim_scores
22889,Behind Enemy Lines,0.447214
1494,The Lost World: Jurassic Park,0.400000
1932,Dune,0.400000
8739,Gamera vs. Jiger,0.400000
10212,Godzilla: Final Wars,0.400000
18193,Prisoners of the Lost Universe,0.400000
22792,Gamera vs. Viras,0.400000
22906,Gamera vs. Guiron,0.400000
23948,Guardians of the Galaxy,0.400000


In [27]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

,title,sim_scores
12589,The Dark Knight,0.800000
10210,Batman Begins,0.700000
9311,Shiner,0.478091
9874,Amongst Friends,0.424264
7772,Mitchell,0.422577
516,Romeo Is Bleeding,0.400000
11463,The Prestige,0.400000
24090,Quicksand,0.400000
25038,Deadfall,0.400000


In [29]:
get_recommendations('The Godfather', cosine_sim2)

,title,sim_scores
1934,The Godfather: Part III,0.527046
1199,The Godfather: Part II,0.421637
15609,The Rain People,0.408248
18940,Last Exit,0.384900
34488,Rege,0.384900
35802,Manuscripts Don't Burn,0.384900
35803,Manuscripts Don't Burn,0.384900
8001,The Night of the Following Day,0.377964
18261,The Son of No One,0.377964
